In [0]:
# %sql
# drop table fq_dev_catalog.silver.dim_store

In [0]:
%sql
CREATE TABLE fq_dev_catalog.silver.dim_store
(
    sys_id STRING NOT NULL,

    store_id INT,
    deployment_name STRING NOT NULL,

    brand_id STRING NOT NULL,
    subsidiary_id INT NOT NULL,
    company_id STRING NOT NULL,

    country_code STRING NOT NULL,
    city STRING NOT NULL,
    store_type STRING NOT NULL,
    zone STRING NOT NULL,

    is_active BOOLEAN NOT NULL,
    source_system STRING NOT NULL,

    store_open_date STRING,

    netsuite_location_name STRING,
    netsuite_location_id INT,

    CONSTRAINT pk_deployment_name PRIMARY KEY (deployment_name)
)
USING DELTA
TBLPROPERTIES (
    delta.enableChangeDataFeed = true,
    delta.columnMapping.mode = 'name'
) 


In [0]:
%sql
select * from fq_dev_catalog.silver.dim_store

In [0]:
staging="abfss://staging@fqadfstoragedev.dfs.core.windows.net/"
df_store_master=spark.read.csv(staging+"master_data/store_master/",header=True,inferSchema=True)
df_store_master.display()


In [0]:
df_store_master.display()

In [0]:
from pyspark.sql.functions import expr, col, when

df_filled = (
    df_store_master
    .withColumn(
        "sys_id",
        when(col("sys_id").isNull(), expr("uuid()"))
        .otherwise(col("sys_id"))
    )
)


In [0]:
df_filled.display()

In [0]:
df_cleaned=(df_filled.withColumnRenamed("subsidiaryId","subsidiary_id")
            .withColumnRenamed("Store Open Date","store_open_date")
            .withColumnRenamed("Netsuite Location Name","netsuite_location_name")
            .withColumnRenamed("Netsuite Location ID","netsuite_location_id")
            .withColumnRenamed("store_name","deployment_name")
)


In [0]:
df_cleaned.display()

In [0]:
df_cleaned.printSchema()

In [0]:
df_cleaned.write \
    .mode("overwrite") \
    .format("delta") \
    .option("mergeSchema","true")\
    .saveAsTable("fq_dev_catalog.silver.dim_store")


In [0]:
%sql
DESCRIBE fq_dev_catalog.silver.dim_store;

In [0]:
%sql
show create table fq_dev_catalog.silver.dim_store

In [0]:
%sql
select * from fq_dev_catalog.silver.dim_store

In [0]:
%sql
update fq_dev_catalog.silver.dim_store
set netsuite_location_name= "ALB-102-Mall of the Emirates"
where netsuite_location_id =113

In [0]:
%sql
ALTER TABLE fq_dev_catalog.silver.dim_store DROP CONSTRAINT pk_deployment_name;

In [0]:
%sql
describe table fq_dev_catalog.silver.dim_store